In [ ]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas
import question
import random

In [2]:
# Predicates
SUBCLASS_OF = "P279"
PART_OF = "P361"
INSTANCE_OF = "P31"

predicates = [SUBCLASS_OF, PART_OF, INSTANCE_OF]

In [45]:
# Category Example: Team Sports
# Element Example: Baseball
from disambiguate import Disambiguate

class Generator:
    def __init__(self, sister_predicates):
        self.sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
        self.sister_predicates = sister_predicates
        self.disambiguater = Disambiguate()
    
    # SPARQL Helper Methods
    def run_query(self, query):
        '''
        Description:
            Takes in query and requests its output
        
        Arguments:
            query:string
        
        Returns:
            results:JSON
        '''
        # Set the query and the return format (JSON)
        self.sparql.setQuery(query)
        self.sparql.setReturnFormat(JSON)

        # Perform the query and convert the result to a Python dictionary
        results = self.sparql.query().convert()
        return results 
    def find_label_by_ID(self, ID):
        '''
        Description:
            Takes in QID and outputs its label
        
        Arguments:
            ID:string - "QXXXXX"
        
        Returns:
            label:string
        '''
        query = f'''
        SELECT ?itemLabel WHERE {{
            wd:{ID} rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en")
        }}
        '''
        results = self.run_query(query)
        return results["results"]["bindings"][0]["itemLabel"]["value"]
    def find_uri_by_label(self, label):
        '''
        Description:
            Takes in label and outputs its URI
        
        Arguments:
            label:string
        
        Returns:
            uri:string - 'http://www.wikidata.org/entity/XXXXXXX'
        '''
        
        # Create SPARQL query to find the URI for a given label
        query = f'''SELECT ?item WHERE {{ 
                    ?item rdfs:label "{label.replace('"', '\"')}"@en.
                    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }} 
                }} LIMIT 1'''
        
        try:
            results = self.run_query(query)

            # The first matching URI
            binding = results["results"]["bindings"][0]
            result = binding["item"]["value"]
            return result
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    def find_ID_by_label(self, label, user_input=False):
        '''
        Description:
            Takes in label and outputs its ID
        
        Arguments:
            label:string
            user_input:bool - True if the label is from the user
        
        Returns:
            id:string - 'QXXXXXX'
        '''
        if user_input:
            id = self.disambiguater.getCollectAnswerQid(label)[1]
        else:
            uri = self.find_uri_by_label(label)
            id = uri.split("/")[-1]
        return id
    def find_ID_by_uri(self, uri):
        '''
        Description:
            Takes in uri and outputs its ID
        
        Arguments:
            uri:string - http://www.wikidata.org/entity/QXXXXX or http://www.wikidata.org/prop/direct/PXXXXX 
        
        Returns:
            id:string - 'Q/PXXXXXX'
        '''
        return uri.split('/')[-1]
    def process_JSON(self, results_JSON, desired_variables):
        '''
        Description:
            Takes in output of a query and returns the desired outputs
        
        Arguments:
            results_JSON:JSON
            desired_variables:List(string) - variables wanted from output 
        
        Returns:
            results:List(ID)
        '''
        results = []
        for c in results_JSON['results']['bindings']:
            r = []
            for variable in desired_variables:
                r.append(self.find_ID_by_uri(c[variable]['value']))
            results.append(r)

        return results
    def random_items(self, items, count):
        '''
        Description:
            Takes in a list of items and outputs a random subset of them
        
        Arguments:
            items:list
            count:int
        
        Returns:
            selected:list() - List of count items
        '''
        
        selected = random.sample(items, count)
        return selected

    def sister_category(self, categoryID, avoided_elementIDs, included_elementIDs=[], n_items=3):
        '''
        Description:
            Takes in category and element IDs and outputs related categories that don't contain the element
        
        Arguments:
            categoryID:string
            avoided_elementIDs:list(string) - elements the category shouldn't cover
            included_elementIDs:list(string) - elements the category should cover - unimplemented
            n_items:int
        
        Returns:
            results:List(ID) - IDs of Sister categories of categoryID
            preds:List(ID) - IDS of predicates corresponding to results
            
        '''
        # Prepare elements
        avoided_element_queries = ""
        for elementID in avoided_elementIDs:
            avoided_element_queries += f'wd:{elementID} ?predicates ?category.\n'
        
        # included_element_queries = ""
        # if len(included_elementIDs) != 0:
        #     # included_element_queries += "VALUES ?includedElements {"
        #     for elementID in included_elementIDs:
        #         included_element_queries += f'wd:{elementID} '
        #     # included_element_queries += "}\n"
        #     # included_element_queries += "?includedElements ?predicates ?category.\n"
            


        # Make query
        desired_variables = ["category", "predicate"]
        query = f'''
                SELECT ?category ?categoryLabel ?predicate WHERE {{
                    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
                    VALUES ?predicates {{ {self.sister_predicates} }}
                    wd:{categoryID} ?predicates ?superCategory.
                    ?category ?predicates ?superCategory.

                    #VALUES ?includedElements {{ included_element_queries }}
                    #?includedElements ?predicates ?category.

                    FILTER(?category != wd:{categoryID})
                    FILTER NOT EXISTS {{
                        {avoided_element_queries}
                    }}
                
                    ?superCategory rdfs:label ?superCategoryLabel .
                    FILTER(LANG(?superCategoryLabel) = "en")
                    ?category rdfs:label ?categoryLabel .
                    FILTER(LANG(?categoryLabel) = "en")
                    BIND(?predicates AS ?predicate)
  
                }}
                LIMIT {n_items}
                #GROUP BY ?category ?categoryLabel
                #HAVING (COUNT(?includedElements) = {len(included_elementIDs)})
        '''
        
        results = self.run_query(query)
        
        results = self.process_JSON(results, desired_variables)
        return [r[0] for r in results], [r[1] for r in results]
    def sister_element(self, elementID, categoryID, exceptions=[], n_items=100):
        '''
        Description:
            Takes in element's ID and its category's ID and outputs related elements not in said category
        
        Arguments:
            elementID:string - "QXXXXX"
            categoryID:string - "QXXXXX"
            exceptions: list 
            n_items:int
        
        Returns:
            results:List(ID) - IDs of Sister topics of label
        '''
        
        #exeptions = [f'FILTER NOT EXISTS {{?item {exception} .}}\n' for exception in exceptions]
        exception_patterns = []
        for exception in exceptions:
            predicate, object = exception.split()
            pattern = f"""FILTER NOT EXISTS {{
                            ?sisterElement {predicate} ?exceptionCategory .
                            ?exceptionCategory (wdt:P279)* {object} .
                        }}\n"""
            exception_patterns.append(pattern)

        exceptions_string = "".join(exception_patterns)

        # Get Sister Catgeories
        sister_categories, sister_preds = self.sister_category(categoryID, [elementID], n_items=n_items)
        sister_preds = list(set(sister_preds))
        # Note: sister_preds[i] represents the predicate used to find sister_categories[i]
        '''
        Example:
        Some superCategory is a category that categoryID is within based on sister_preds[i]
        sister_categories[i] is in superCategory based on sister_preds[i]

        Thus, to find a sister of elementID (more like cousin), the sister_element must be within sister_categories[i] based on sister_preds[i]

        So, let's say the category is team sports.. Team sports is a SUBCLASS_OF sports.
        Animal sports is a SUBCLASS_OF sports
        Horse racing is a SUBCLASS_OF animal sports

        In that example:
        categoryID = team sports
        superCategory = sports
        sister_category[i] = animal sports
        sister_preds[i] = SUBCLASS_OF
        sister of elementID = horse racing

        Using sister_preds ensures that the sister_elements generated are closer to elementID
        '''

        #SPARQL query to find elements that are similar to the elementID but not in categoryID
        desired_variables = ["sisterElement"]
        query = f'''
        SELECT ?sisterElement ?sisterElementLabel WHERE {{
            VALUES ?predicates {{ {"".join([f'wdt:{p} ' for p in sister_preds])} }}
            VALUES ?sisterCategories {{ {"".join([f"wd:{sister} " for sister in sister_categories])} }}
            
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            ?sisterElement ?predicates ?sisterCategories.
            ?sisterElement rdfs:label ?sisterElementLabel.

            FILTER(LANG(?sisterElementLabel) = "en")
            FILTER(?sisterElement != wd:{elementID})
            {exceptions_string}
            
        }}
        
        LIMIT {n_items}
        '''
        
        results = self.run_query(query)
        results = self.process_JSON(results, desired_variables)

        return [r[0] for r in results]
        
    
    # Printing Methods
    def display_as_table(self, results, n_items):
        '''
        Description:
            Creates a table of the n_items of the queried results
        
        Arguments:
            results:JSON - sister topics
            n_items:int
        
        Returns:
            None
        '''
        df = pandas.DataFrame.from_dict(results["results"]["bindings"][:n_items])
        df = df.applymap(lambda x: x["value"])
        pandas.set_option('display.max_rows', n_items) # n_items doesnt work here
        print(df)
    def print_question(self, question):
        '''
        Description:
            Prints the question
        
        Arguments:
            question:Question
        
        Returns:
            None
        '''
        print(f"Which of the following is a {question.relation.predicate} {question.relation.object}?")
       
        print(question.all_answers)
        for index, answer in enumerate(question.all_answers):
            print(f"{index + 1}. {self.find_label_by_ID(answer)}")
   
    # Element-> Category+Elements Question Methods
    '''
    Which of the following is a subclass of Olympic sport?
    1. savika
    2. *baseball*
    3. memory sport
    4. snowshoe biathlon
    '''
    def get_category_relation(self, label):
        '''
        Description:
            Takes in label and outputs one of its categories in the form of a relation
            Relations are pairs of predicates and objects

        Arguments:
            label:string
        
        Returns:
            Relation object
        '''
        topicID = self.find_ID_by_label(label)
        # Randomly choose a ?predicate ?object pair
        desired_variables = ["predicate", "object"]
        query = f'''
        SELECT ?predicate ?object WHERE {{
            VALUES ?predicate {{ {self.sister_predicates} }}
            
            ?object rdfs:label ?label.
            wd:{topicID} ?predicate ?object.
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
        }}
        '''
        results = self.process_JSON(self.run_query(query), desired_variables)
        random_relation = self.random_items(results, count=1)[0]

        return question.Relation(random_relation[0], random_relation[1])
    def element_question(self, element_label, num_wrong_answers=3):
        '''
        Description:
           Creates a question with the element as the correct answer
        
        Arguments:
            element_label:string
            num_wrong_answers:int
        
        Returns:
            question:Question
        '''
        elementID = self.find_ID_by_label(element_label)
        
        relation = self.get_category_relation(element_label)
        categoryID = relation.object
        wrong_answers = self.sister_element(elementID, categoryID, [str(relation)])

        selected_answers = self.random_items(wrong_answers, count=num_wrong_answers)
        relation = question.Relation( self.find_label_by_ID(relation.predicate), self.find_label_by_ID(relation.object))
        return question.Question(relation, elementID, selected_answers)
    
    # Element(s) -> Categories Question Methods
    '''
    Which of the following is a category baseball, volleyball, basketball, and cricket all belong to?
    1. sport in ancient Greece
    2. *team sport*
    3. racing sports
    4. sport in Europe
    '''
    def find_category(self, elementIDs, predicateID=SUBCLASS_OF):
        '''
        Description:
            Inputs element IDs and outputs a category ID that they all belong to based on the predicate
        
        Arguments:
            elementIDs:list(string)
        
        Returns:
            random_categoryID:string -  random category that elementID belongs to
            predicateID:string - predicate that links the element and the category
        '''
        # Prepare elements
        element_queries = "".join([f"wd:{e} " for e in elementIDs]) # wd:QXXXX for every element
        
        # Prepare query
        desired_variables = ["category", "predicate"]
        query = f'''
        SELECT ?category ?label ?predicate (COUNT(?elements) as ?elementCount) WHERE {{
            VALUES ?predicates {{ {self.sister_predicates} }}
    
            VALUES ?elements {{ {element_queries} }}
            #?elements wdt:{predicateID} ?category.
            ?elements ?predicates ?category.
            ?category rdfs:label ?label.
            BIND(?predicates AS ?predicate)

            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
            FILTER(LANG(?label) = "en")
        }}
        GROUP BY ?category ?label ?predicate
        HAVING (COUNT(?elements) = {len(elementIDs)})
        '''
           
        categories = self.process_JSON(self.run_query(query), desired_variables)
        if len(categories) == 0:
            print(query)
            raise Exception("These elements do not have a shared category")
        
        random_category = self.random_items(categories, count=1)[0]
        random_categoryID = random_category[0]
        predicateID = random_category[1]
        
        return random_categoryID, predicateID
    def category_question(self, element_labels, num_wrong_answers=3):
        '''
        Description:
            Inputs elements and outputs 4 categories, one being the one that contains every element
        
        Arguments:
            element_labels:list(string)
            num_wrong_answers:int
        
        Returns:
            question:Question
        '''
        # Get category of the elements
        elementIDs = []
        for element_label in element_labels:
            elementIDs.append(self.find_ID_by_label(element_label, user_input=True))
        categoryID, predicateID = self.find_category(elementIDs) 
        
        # Get 3 other sister categories that are similar, but not the correct answer
        results, _ = self.sister_category(categoryID, elementIDs, n_items=num_wrong_answers) 

        # Prepare object of relation
        object = ""
        i=0
        while i < len(element_labels) - 1:
            object += f"{element_labels[i]}, "
            i+=1
        
        if len(element_labels) == 1:
            object += f"{element_labels[i]} belongs to"
        else:
            object += f"and {element_labels[i]} all belong to"

        # Return the 3 sisters and the correct category
        relation = question.Relation("category", object)
        
        return question.Question(relation, categoryID, results)


In [46]:
named_after = "wdt:P138"
occupation = "wdt:P106"
has_use = "wdt:P366"
studied_in = "wdt:P2579"
sensible_sister_predicates = f"wdt:P279 wdt:P361 wdt:P101 wdt:P361 wdt:P921 wdt:P131 wdt:P150 {named_after} {occupation} {has_use} {studied_in}"

generator = Generator(sister_predicates=sensible_sister_predicates)

In [6]:
import time
def question_answer(question):
    generator.print_question(question)
    time.sleep(0.2)

    answer = int(input("What is your answer?")) - 1
    user_answer = generator.find_label_by_ID(question.all_answers[answer])
    print(f"Your answer was {user_answer}")
    correct_answer = generator.find_label_by_ID(question.correct_answer)
    print(f"The correct answer was {correct_answer}")
    if user_answer == correct_answer:
        print("You are correct!")
    else:
        print("You are incorrect!")

In [70]:
# 1 Input
cat_q = generator.category_question(["mustard"])
question_answer(cat_q)

- Multiple Items came up when you searched ' mustard , Did you mean:
- 1 condiment made from various varieties of mustard seeds
- 2 species of mustard plant
- 3 American record producer and DJ
- 4 species of plant
- 5 chemical compound
- 6 family name
- 7 genus of plants
- 8 plants used for mustard
- 9 type of mustard
- 10 species of plant
The user input is: 1
The selected answer: condiment made from various varieties of mustard seeds and the Qid: Q131748

Which of the following is a category mustard belongs to?
['Q8495', 'Q6099', 'Q5726634', 'Q1269']
1. milk
2. escalope
3. food paste
4. jam
Your answer was milk
The correct answer was food paste
You are incorrect!


In [68]:
# 2 Inputs
cat_q = generator.category_question(["Joe Biden", "Donald Trump"])
question_answer(cat_q)

- Multiple Items came up when you searched ' Joe Biden , Did you mean:
- 1 President of the United States since 2021
- 2 fictional parody character from The Onion loosely based on the real Joe Biden
- 3 2020 United States presidential campaign of Joe Biden
- 4 Wikimedia category
- 5 U.S. presidential admistration since 2021
- 6 Members of United States President Joe Biden&#039;s Cabinet
- 7 2024 United States presidential campaign of Joe Biden
- 8 59th United States presidential inauguration
- 9 the inaugural address of Joe Biden
- 10 List of elections featuring Joe Biden as a candidate
The user input is: 1
The selected answer: President of the United States since 2021 and the Qid: Q6279

- Multiple Items came up when you searched ' Donald Trump , Did you mean:
- 1 president of the United States from 2017 to 2021
- 2 American physician
- 3 American businessman and reality television personality
- 4 first impeachment inquiry against U.S. President Donald Trump
- 5 son of Donald Trump Jr

In [69]:
# 3 Inputs
cat_q = generator.category_question(["dog", "cat", "cow"])
question_answer(cat_q)

- Multiple Items came up when you searched ' dog , Did you mean:
- 1 domestic animal
- 2 dog musher
- 3 working dog
- 4 functional type of dog
- 5 group of closely related and visibly similar domestic dogs
- 6 American educational computer animated television series
- 7 American video game developer
- 8 Category 5 Atlantic hurricane in 1950
- 9 sign of Chinese zodiac
- 10 2022 film directed by Reid Carolin and Channing Tatum
The user input is: 1
The selected answer: domestic animal and the Qid: Q144

- Multiple Items came up when you searched ' cat , Did you mean:
- 1 domesticated feline
- 2 medical imaging procedure using X-rays to produce cross-sectional images
- 3 Museu Paulista&#039;s collection
- 4 use with &#039;instance of&#039; (P31) for Wikimedia category
- 5 Unix utility that concatenates and lists files
- 6 fictional character in Marvel Comics
- 7 cat and Internet meme celebrity
- 8 human disease
- 9 British musician (born 1948)
- 10 cat breed
The user input is: 1
The select

In [ ]:
my_question = generator.element_question("baseball", 3)
question_answer(my_question)

In [ ]:
my_question = generator.element_question("anteater", 3)
question_answer(my_question)

In [ ]:

def get_wikidata_item(item_id):
    # Wikidata API endpoint
    url = "https://www.wikidata.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "wbgetentities",  # Action to get data about entities
        "ids": item_id,  # ID of the Wikidata item (e.g., Q64 for Berlin)
        "format": "json",  # Response format
        "props": "labels|descriptions|aliases",  # Properties to retrieve: labels, descriptions, and aliases
        "languages": "en"  # Language filter
    }
    
    # Make the GET request to the Wikidata API
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the item's data
        item_data = data['entities'][item_id]
        
        # Extract and print the label, description, and aliases
        label = item_data['labels']['en']['value']
        description = item_data['descriptions']['en']['value']
        aliases = [alias['value'] for alias in item_data['aliases']['en']]
        
        print(f"Label: {label}")
        print(f"Description: {description}")
        print(f"Aliases: {', '.join(aliases)}")
    else:
        print("Failed to retrieve data")

# Example usage
get_wikidata_item("Q64")